# Introduction

In this notebook, I study the generation of a ZIGZAG indicator based on BollingerBands indicator with Multi-Timeframe capabilities.

In [10]:
import sys

####################################################################################
# Data handling 
import pandas as pd
from pandas import concat
from pandas.plotting import scatter_matrix
import numpy as np

####################################################################################
# Visualization
import matplotlib.pyplot as plt
from matplotlib import dates, ticker
from matplotlib.dates import (MONDAY, DateFormatter, MonthLocator, WeekdayLocator, date2num)
import matplotlib as mpl
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import plotly.tools as tls
plotly.offline.init_notebook_mode(connected=True)

####################################################################################
# TA-Lib: 
import talib

####################################################################################
# Other utilities
import datetime
import time
import os
import sys
import math
from enum import Enum
import logging
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

print('Packages loaded!!')

Packages loaded!!


In [11]:
# Append path for MACD_Signal_Listener class (parent folder)
sys.path.append('..')

# import main class and event class
from ZIGZAG_Signal_Listener import ZIGZAG_Signal_Listener

print('Done!')

Done!


In [6]:
# creates zigzag signal listener
zz = ZIGZAG_Signal_Listener(level=logging.DEBUG)
zz

INFO:ZIGZAG_Signal_Listener:Created!


## Configurations

### 1) One indicator per timeframe

In [17]:
# loads df dataframe at M15, H1 and H4
df_M15 = pd.read_csv('../csv_data/EURUSD_M15.csv', sep=';')
df_M15['TIME'] = df_M15['DATE'] + '  ' + df_M15['TIME'] 
df_M15['TIME'] = df_M15['TIME'].map(lambda x: datetime.datetime.strptime(x, '%Y.%m.%d %H:%M:%S'))  
df_M15['TIME'] = pd.to_datetime(df_M15['TIME'])
df_M15 = df_M15[['TIME', 'OPEN','HIGH','LOW','CLOSE']]

df_H1 = pd.read_csv('../csv_data/EURUSD_H1.csv', sep=';')
df_H1['TIME'] = df_H1['DATE'] + '  ' + df_H1['TIME'] 
df_H1['TIME'] = df_H1['TIME'].map(lambda x: datetime.datetime.strptime(x, '%Y.%m.%d %H:%M:%S'))  
df_H1['TIME'] = pd.to_datetime(df_H1['TIME'])
df_H1 = df_H1[['TIME', 'OPEN','HIGH','LOW','CLOSE']]

df_H4 = pd.read_csv('../csv_data/EURUSD_H4.csv', sep=';')
df_H4['TIME'] = df_H4['DATE'] + '  ' + df_H4['TIME'] 
df_H4['TIME'] = df_H4['TIME'].map(lambda x: datetime.datetime.strptime(x, '%Y.%m.%d %H:%M:%S'))  
df_H4['TIME'] = pd.to_datetime(df_H4['TIME'])
df_H4 = df_H4[['TIME', 'OPEN','HIGH','LOW','CLOSE']]

print('m15_rows={}, h1_rows={}, h4_rows={}'.format(df_M15.shape[0],df_H1.shape[0],df_H4.shape[0]))

m15_rows=109028, h1_rows=27264, h4_rows=6825


In [18]:
# builds indicators for each timeframe
_df_H4_result, _events = zz.ZIGZAG( df_H4, 
                                    minbars=12, 
                                    bb_period=20, 
                                    bb_dev = 2.0,
                                    bb_sma = [100],
                                    nan_value = 0.0, 
                                    level=logging.WARN)
_df_H1_result, _events = zz.ZIGZAG( df_H1, 
                                    minbars=12, 
                                    bb_period=20, 
                                    bb_dev = 2.0,
                                    bb_sma = [100],
                                    nan_value = 0.0, 
                                    level=logging.WARN)
_df_M15_result, _events = zz.ZIGZAG(df_M15, 
                                    minbars=12, 
                                    bb_period=20, 
                                    bb_dev = 2.0,
                                    bb_sma = [100],
                                    nan_value = 0.0, 
                                    level=logging.WARN)

print('M15 from={} to={}\r\nH1  from={} to={}\r\nH4  from={} to={}'.format(_df_M15_result.TIME.iloc[0], 
                                                                          _df_M15_result.TIME.iloc[-1], 
                                                                          _df_H1_result.TIME.iloc[0], 
                                                                          _df_H1_result.TIME.iloc[-1], 
                                                                          _df_H4_result.TIME.iloc[0], 
                                                                          _df_H4_result.TIME.iloc[-1]))
                                                                          

KeyboardInterrupt: 

In [ ]:
_df_H4_result.tail(20)

In [17]:
# setup same starting date
same_time = max(_df_M15_result.TIME.iloc[0], _df_H1_result.TIME.iloc[0], _df_H4_result.TIME.iloc[0])

# filter samples previous same_time and reset index to start from 0
_df_M15_result = _df_M15_result[_df_M15_result.TIME >= same_time].reset_index(drop=True)
_df_H1_result = _df_H1_result[_df_H1_result.TIME >= same_time].reset_index(drop=True)
_df_H4_result = _df_H4_result[_df_H4_result.TIME >= same_time].reset_index(drop=True)

print('M15 from={} to={}\r\nH1  from={} to={}\r\nH4  from={} to={}'.format(_df_M15_result.TIME.iloc[0], 
                                                                          _df_M15_result.TIME.iloc[-1], 
                                                                          _df_H1_result.TIME.iloc[0], 
                                                                          _df_H1_result.TIME.iloc[-1], 
                                                                          _df_H4_result.TIME.iloc[0], 
                                                                          _df_H4_result.TIME.iloc[-1]))
                                                                          

M15 from=2015-01-29 20:00:00 to=2019-05-23 00:00:00
H1  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00
H4  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00


In [18]:
# Plot H4 zigzag indicator
trace_ohlc = go.Ohlc(x=_df_H4_result.index.values, open=_df_H4_result.OPEN, high=_df_H4_result.HIGH, low=_df_H4_result.LOW, close=_df_H4_result.CLOSE, name='Candlestick')

_dfz_h4 = _df_H4_result[_df_H4_result.ZIGZAG > 0].copy()
trace_zigzag = go.Scatter(x=_dfz_h4.reset_index()['index'], y=_dfz_h4.ZIGZAG, name='h4', line=scatter.Line(color='black', width=1))

py.iplot([trace_ohlc, trace_zigzag], filename='simple_ohlc')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [19]:
# Plot H1 zigzag indicator
trace_ohlc = go.Ohlc(x=_df_H1_result.index.values, open=_df_H1_result.OPEN, high=_df_H1_result.HIGH, low=_df_H1_result.LOW, close=_df_H1_result.CLOSE, name='Candlestick')

_dfz_h1 = _df_H1_result[_df_H1_result.ZIGZAG > 0].copy()
trace_zigzag = go.Scatter(x=_dfz_h1.reset_index()['index'], y=_dfz_h1.ZIGZAG, name='h1', line=scatter.Line(color='black', width=1))

py.iplot([trace_ohlc, trace_zigzag], filename='simple_ohlc')



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [20]:
# Plot M15 zigzag indicator limited to 20k
_df_M15_limit = _df_M15_result[0:20000].copy()
trace_ohlc = go.Ohlc(x=_df_M15_limit.index.values, open=_df_M15_limit.OPEN, high=_df_M15_limit.HIGH, low=_df_M15_limit.LOW, close=_df_M15_limit.CLOSE, name='Candlestick')

_dfz_M15 = _df_M15_limit[_df_M15_limit.ZIGZAG > 0].copy()
trace_zigzag_M15 = go.Scatter(x=_dfz_M15.reset_index()['index'], y=_dfz_M15.ZIGZAG, name='h1', line=scatter.Line(color='black', width=1))

py.iplot([trace_ohlc, trace_zigzag_M15], filename='simple_ohlc')



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [21]:
# plot zigzag at H4 and H1
trace_zigzag_H4 = go.Scatter(x=_dfz_h4.reset_index()['index']*4, y=_dfz_h4.ZIGZAG, name='h4', line=scatter.Line(color='black', width=1))
trace_zigzag_H1 = go.Scatter(x=_dfz_h1.reset_index()['index'], y=_dfz_h1.ZIGZAG, name='h1', line=scatter.Line(color='red', width=1))

py.iplot([trace_zigzag_H4, trace_zigzag_H1], filename='simple_ohlc')



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [26]:
# plot zigzag at H1 and M15 limited to 20k

_df_M15_limit = _df_M15_result[0:20000].copy()
_dfz_M15 = _df_M15_limit[_df_M15_limit.ZIGZAG > 0].copy()
trace_zigzag_M15 = go.Scatter(x=_dfz_M15.reset_index()['index'], y=_dfz_M15.ZIGZAG, name='m15', line=scatter.Line(color='red', width=1))

_df_H1_limit = _df_H1_result[0:5000].copy()
_dfz_H1 = _df_H1_limit[_df_H1_limit.ZIGZAG > 0].copy()
trace_zigzag_H1 = go.Scatter(x=_dfz_H1.reset_index()['index']*4, y=_dfz_H1.ZIGZAG, name='h1', line=scatter.Line(color='black', width=1))

py.iplot([trace_zigzag_H1, trace_zigzag_M15], filename='simple_ohlc')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [27]:
# plot zigzag at H4 and M15 limited to 20k

_df_M15_limit = _df_M15_result[0:20000].copy()
_dfz_M15 = _df_M15_limit[_df_M15_limit.ZIGZAG > 0].copy()
trace_zigzag_M15 = go.Scatter(x=_dfz_M15.reset_index()['index'], y=_dfz_M15.ZIGZAG, name='m15', line=scatter.Line(color='red', width=1))

_df_H4_limit = _df_H4_result[0:1250].copy()
_dfz_H4 = _df_H4_limit[_df_H4_limit.ZIGZAG > 0].copy()
trace_zigzag_H4 = go.Scatter(x=_dfz_H4.reset_index()['index']*16, y=_dfz_H4.ZIGZAG, name='h4', line=scatter.Line(color='black', width=1))

py.iplot([trace_zigzag_H4, trace_zigzag_M15], filename='simple_ohlc')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### 2) Sinthetic indicators from smaller timeframes

**Build H4 from H1**

In [28]:
# Build a sinthetic zigzag indicator at H1 for H4
_df_H4S_result, _x, _y, _events = zz.ZIGZAG(df_H1, 
                                  minbars=48, 
                                  bb_period=80, 
                                  bb_dev = 2.0,
                                  bb_sma = [100],
                                  nan_value = 0.0, 
                                  level=logging.WARN)

print('H4S  from={} to={}\r\nH4  from={} to={}'.format(_df_H4S_result.TIME.iloc[0], 
                                                       _df_H4S_result.TIME.iloc[-1], 
                                                       _df_H4_result.TIME.iloc[0], 
                                                       _df_H4_result.TIME.iloc[-1]))

H4S  from=2015-01-13 18:00:00 to=2019-05-23 00:00:00
H4  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00


In [29]:
# setup same starting date
same_time = max(_df_H4S_result.TIME.iloc[0], _df_H4_result.TIME.iloc[0])

# filter samples previous same_time and reset index to start from 0
_df_H4S_result = _df_H4S_result[_df_H4S_result.TIME >= same_time].reset_index(drop=True)
_df_H4_result = _df_H4_result[_df_H4_result.TIME >= same_time].reset_index(drop=True)

print('H4S  from={} to={}\r\nH4  from={} to={}'.format(_df_H4S_result.TIME.iloc[0], 
                                                       _df_H4S_result.TIME.iloc[-1], 
                                                       _df_H4_result.TIME.iloc[0], 
                                                       _df_H4_result.TIME.iloc[-1]))

H4S  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00
H4  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00


In [32]:
# Plot H4 and H4Sinthetic zigzag indicators

trace_ohlc = go.Ohlc(x=_df_H1_result.index.values, open=_df_H1_result.OPEN, high=_df_H1_result.HIGH, low=_df_H1_result.LOW, close=_df_H1_result.CLOSE, name='Candlestick')

_dfz_h4 = _df_H4_result[_df_H4_result.ZIGZAG > 0].copy()
trace_zigzag_h4 = go.Scatter(x=_dfz_h4.reset_index()['index']*4, y=_dfz_h4.ZIGZAG, name='h4', line=scatter.Line(color='black', width=1))

_dfz_h4s = _df_H4S_result[_df_H4S_result.ZIGZAG > 0].copy()
trace_zigzag_h4s = go.Scatter(x=_dfz_h4s.reset_index()['index'], y=_dfz_h4s.ZIGZAG, name='h4_sinthetic', line=scatter.Line(color='red', width=1))

py.iplot([trace_ohlc, trace_zigzag_h4, trace_zigzag_h4s], filename='simple_ohlc')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


**Build H4 from M15**

In [33]:
# Build a sinthetic zigzag indicator at M15 for H4
_df_H4S_result, _x, _y, _events = zz.ZIGZAG(df_M15, 
                                  minbars=192, 
                                  bb_period=320, 
                                  bb_dev = 2.0,
                                  bb_sma = [100],
                                  nan_value = 0.0, 
                                  level=logging.WARN)

print('H4S  from={} to={}\r\nH4  from={} to={}'.format(_df_H4S_result.TIME.iloc[0], 
                                                       _df_H4S_result.TIME.iloc[-1], 
                                                       _df_H4_result.TIME.iloc[0], 
                                                       _df_H4_result.TIME.iloc[-1]))

H4S  from=2015-01-08 17:15:00 to=2019-05-23 00:00:00
H4  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00


In [34]:
# setup same starting date
same_time = max(_df_H4S_result.TIME.iloc[0], _df_H4_result.TIME.iloc[0])

# filter samples previous same_time and reset index to start from 0
_df_H4S_result = _df_H4S_result[_df_H4S_result.TIME >= same_time].reset_index(drop=True)
_df_H4_result = _df_H4_result[_df_H4_result.TIME >= same_time].reset_index(drop=True)

print('H4S  from={} to={}\r\nH4  from={} to={}'.format(_df_H4S_result.TIME.iloc[0], 
                                                       _df_H4S_result.TIME.iloc[-1], 
                                                       _df_H4_result.TIME.iloc[0], 
                                                       _df_H4_result.TIME.iloc[-1]))

H4S  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00
H4  from=2015-01-29 20:00:00 to=2019-05-23 00:00:00


In [35]:
# Plot H4 and H4Sinthetic zigzag indicators, as built from M15, limit to 20k samples
_df_M15_limit = _df_M15_result[0:20000].copy()
trace_ohlc = go.Ohlc(x=_df_M15_limit.index.values, open=_df_M15_limit.OPEN, high=_df_M15_limit.HIGH, low=_df_M15_limit.LOW, close=_df_M15_limit.CLOSE, name='Candlestick')

_df_H4_limit = _df_H4_result[0:1250].copy()
_dfz_h4 = _df_H4_limit[_df_H4_limit.ZIGZAG > 0].copy()
trace_zigzag_h4 = go.Scatter(x=_dfz_h4.reset_index()['index']*16, y=_dfz_h4.ZIGZAG, name='h4', line=scatter.Line(color='black', width=1))

_df_H4S_limit = _df_H4S_result[0:20000].copy()
_dfz_h4s = _df_H4S_limit[_df_H4S_limit.ZIGZAG > 0].copy()
trace_zigzag_h4s = go.Scatter(x=_dfz_h4s.reset_index()['index'], y=_dfz_h4s.ZIGZAG, name='h4_sinthetic', line=scatter.Line(color='red', width=1))

py.iplot([trace_ohlc, trace_zigzag_h4, trace_zigzag_h4s], filename='simple_ohlc')


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


# Conclussions

This zigzag indicator, as shown above, is able to build upwards timeframe indicators from a smaller one, just modifying ```minbars``` and ```bb_period ``` parameters accordingly with the timeframe difference. Some examples:

```
- From M15 calculate M30 zigzag: minbars = 12*2=24,   bb_period = 20*2 = 40
- From M15 calculate H2 zigzag:  minbars = 12*8=96,   bb_period = 20*8 = 160
- From H1  calculate D1 zigzag:  minbars = 12*24=288, bb_period = 20*24 = 480
```
